# 2_Sql_to_Json_Dataset.ipynb

Este notebook transforma um banco de dados SQLite em um JSON consolidado. Ele considera as tabelas dentro do banco
e gera um arquivo JSON contendo os dados. O campo `gnss_data_json`, que já é um JSON no banco, será embutido
diretamente no JSON final.

## Estrutura do banco de dados:
- Nome do banco: Consolidado.db
- Tabelas:
  - `gnss_data`: Contém dados de latitude, longitude, altitude, e um campo `gnss_data_json` que já é um JSON.

## Objetivo:
- Ler os dados da tabela `gnss_data` e criar um único JSON consolidado contendo todos os registros.
- Salvar o JSON no arquivo `Dataset.json`.

## Saída:
- Um arquivo chamado `Dataset.json` com todos os dados consolidados.

In [1]:
import sqlite3
import json
from pathlib import Path

# Configuração dos caminhos

In [2]:
current_dir = Path().resolve()  # Diretório onde o notebook está sendo executado
base_dir = current_dir.parent / "TABELAS"  # Local onde a pasta TABELAS está localizada
json_output_dir = base_dir / "JSONS"  # Local onde o JSON será salvo
db_path = base_dir / "SQL'S" / "Consolidado.db"  # Caminho para o banco Consolidado.db
output_json_path = json_output_dir / "Dataset.json"  # Caminho para salvar o JSON consolidado



# Função para transformar os dados do banco em JSON

Transforma os dados de um banco SQLite em um arquivo JSON consolidado.
    
    :param db_path: Caminho para o banco de dados SQLite.
    :param output_path: Caminho para salvar o JSON gerado.

In [3]:
def sql_to_json(db_path, output_path):

    # Conexão com o banco de dados
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Lista para armazenar todos os registros como dicionários
    json_data = []

    # Obtém os dados da tabela gnss_data
    cursor.execute("SELECT * FROM gnss_data;")
    columns = [description[0] for description in cursor.description]  # Nome das colunas
    rows = cursor.fetchall()

    for row in rows:
        # Converte a linha em um dicionário
        record = dict(zip(columns, row))
        
        # O campo `gnss_data_json` é um JSON; converte para dicionário e mantém apenas `satellites`
        if "gnss_data_json" in record and record["gnss_data_json"]:
            satellites_data = json.loads(record["gnss_data_json"]).get("satellites", [])
            record["satellites"] = satellites_data
            del record["gnss_data_json"]  # Remove o campo original
        
        # Remove o campo `_id` do registro
        if "_id" in record:
            del record["_id"]
            
        # Adiciona o registro à lista final
        json_data.append(record)
    
    # Salva os dados no arquivo JSON
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=4)

    conn.close()
    print(f"JSON gerado com sucesso em: {output_path}")


# Executa a conversão

In [4]:
sql_to_json(db_path, output_json_path)

JSON gerado com sucesso em: D:\[TCC]\TABELAS\JSONS\Dataset.json
